In [1]:
from joblib import Parallel, delayed
import pandas
import numpy as np
import json
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [2]:
fileTrain = pandas.read_csv('train.csv')
fileTest = pandas.read_csv('test.csv')

In [3]:

features = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO']
features_cat = ['CO_PROVA_CN','CO_PROVA_CH','CO_PROVA_LC','CO_PROVA_MT']

In [4]:
#TRAIN
dataTrain = fileTrain[features]
dataTrain.fillna(value=0, inplace=True)
x_train = dataTrain.iloc[:,:].values
y_train = fileTrain[['NU_NOTA_MT']].fillna(value=0)

categorical_train = fileTrain[features_cat]
categorical_train.fillna(value=0, inplace=True)

   

C:\Users\igorl\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [5]:

#TESTE
dataTest = fileTest[features]
dataTest.fillna(value=0, inplace=True)
x_test = dataTest.iloc[:,:].values

categorical_test = fileTest[features_cat]
categorical_test.fillna(value=0, inplace=True)

C:\Users\igorl\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [6]:
#dummies
cat =  pandas.concat([categorical_train, categorical_test], axis=0, ignore_index=True)
dummies = pandas.get_dummies(cat, drop_first=True,dummy_na=True)

dummies_train = dummies.iloc[:categorical_train.shape[0],:]
dummies_test = dummies.iloc[categorical_train.shape[0]:,:] 


#jutando as features
x_train = np.append(arr = x_train, values = dummies_train.values, axis =1)
x_test = np.append(arr = x_test, values = dummies_test.values, axis =1)

In [7]:
#SCALING + REGRESSOR
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(x_train)


In [8]:
steps = [ ('SVR', SVR(kernel='rbf'))]
param_grid = {'SVR__C': [10, 100, 200, 400]}

estimator = Pipeline(steps=steps)
cv = GridSearchCV(estimator, param_grid, verbose=5, n_jobs=-1, cv=10)
cv.fit(X, y_train)

print(cv.best_params_, cv.best_score_)

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  7.0min finished
C:\Users\igorl\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'SVR__C': 400} 0.9157625711634626


In [ ]:

{'SVR__C': 400} 0.9158416126858605